In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")


In [ ]:
# Basic statistics and data exploration
print("=== PUBLIC DATASET STATISTICS ===")
print(f"Shape: {public_df.shape}")
print("\nDescriptive Statistics:")
print(public_df.describe())

print("\n=== DATA TYPES ===")
print(public_df.dtypes)

print("\n=== MISSING VALUES ===")
print(public_df.isnull().sum())

# Feature engineering - add derived features
public_df['miles_per_day'] = public_df['miles_traveled'] / public_df['trip_duration_days']
public_df['receipts_per_day'] = public_df['total_receipts_amount'] / public_df['trip_duration_days']
public_df['receipt_to_mile_ratio'] = public_df['total_receipts_amount'] / (public_df['miles_traveled'] + 1)  # +1 to avoid division by zero

print("\n=== DERIVED FEATURES ===")
print("Miles per day range:", public_df['miles_per_day'].min(), "to", public_df['miles_per_day'].max())
print("Receipts per day range:", public_df['receipts_per_day'].min(), "to", public_df['receipts_per_day'].max())


In [ ]:
# Analyze reimbursement by trip duration
trip_duration_analysis = public_df.groupby('trip_duration_days').agg({
    'reimbursement': ['count', 'mean', 'std', 'min', 'max'],
    'miles_traveled': 'mean',
    'total_receipts_amount': 'mean'
}).round(2)

print("=== REIMBURSEMENT BY TRIP DURATION ===")
print(trip_duration_analysis)

# Calculate per-day reimbursement rates
per_day_rates = public_df.groupby('trip_duration_days')['reimbursement'].mean() / public_df.groupby('trip_duration_days')['trip_duration_days'].first()
print("\n=== AVERAGE REIMBURSEMENT PER DAY BY TRIP LENGTH ===")
for days, rate in per_day_rates.items():
    print(f"{days} days: ${rate:.2f} per day")


In [ ]:
# Visualize trip duration patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution of trip durations
axes[0,0].hist(public_df['trip_duration_days'], bins=range(1, public_df['trip_duration_days'].max()+2), 
               alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribution of Trip Durations')
axes[0,0].set_xlabel('Trip Duration (Days)')
axes[0,0].set_ylabel('Frequency')

# Average reimbursement by trip duration
trip_means = public_df.groupby('trip_duration_days')['reimbursement'].mean()
axes[0,1].bar(trip_means.index, trip_means.values, alpha=0.7)
axes[0,1].set_title('Average Reimbursement by Trip Duration')
axes[0,1].set_xlabel('Trip Duration (Days)')
axes[0,1].set_ylabel('Average Reimbursement ($)')

# Per-day rates
axes[1,0].bar(per_day_rates.index, per_day_rates.values, alpha=0.7, color='orange')
axes[1,0].set_title('Average Reimbursement Per Day by Trip Length')
axes[1,0].set_xlabel('Trip Duration (Days)')
axes[1,0].set_ylabel('Reimbursement Per Day ($)')
axes[1,0].axhline(y=100, color='red', linestyle='--', label='$100 baseline')
axes[1,0].legend()

# Box plot of reimbursements by trip duration
public_df.boxplot(column='reimbursement', by='trip_duration_days', ax=axes[1,1])
axes[1,1].set_title('Reimbursement Distribution by Trip Duration')
axes[1,1].set_xlabel('Trip Duration (Days)')
axes[1,1].set_ylabel('Reimbursement ($)')

plt.tight_layout()
plt.show()


In [ ]:
# Analyze mileage patterns
print("=== MILEAGE ANALYSIS ===")

# Create mileage bins for analysis
public_df['mileage_bin'] = pd.cut(public_df['miles_traveled'], 
                                  bins=[0, 50, 100, 200, 400, 600, 800, 1000, float('inf')],
                                  labels=['0-50', '51-100', '101-200', '201-400', '401-600', '601-800', '801-1000', '1000+'])

mileage_analysis = public_df.groupby('mileage_bin').agg({
    'reimbursement': ['count', 'mean', 'std'],
    'miles_traveled': 'mean',
    'trip_duration_days': 'mean',
    'total_receipts_amount': 'mean'
}).round(2)

print(mileage_analysis)

# Calculate implied mileage rates
# Estimate base per diem (assume $100/day as baseline)
public_df['estimated_per_diem'] = public_df['trip_duration_days'] * 100
public_df['estimated_mileage_component'] = public_df['reimbursement'] - public_df['estimated_per_diem']
public_df['implied_mileage_rate'] = public_df['estimated_mileage_component'] / public_df['miles_traveled']

# Remove outliers for cleaner analysis
mileage_rates = public_df[
    (public_df['implied_mileage_rate'] > 0) & 
    (public_df['implied_mileage_rate'] < 2.0) &
    (public_df['miles_traveled'] > 0)
].copy()

print(f"\n=== IMPLIED MILEAGE RATES (after filtering) ===")
print(f"Sample size: {len(mileage_rates)} cases")
print(f"Mean rate: ${mileage_rates['implied_mileage_rate'].mean():.3f} per mile")
print(f"Median rate: ${mileage_rates['implied_mileage_rate'].median():.3f} per mile")
print(f"Std dev: ${mileage_rates['implied_mileage_rate'].std():.3f}")

# Analyze rates by mileage ranges
mileage_rates['mileage_range'] = pd.cut(mileage_rates['miles_traveled'], 
                                       bins=[0, 100, 200, 400, 600, 1000, float('inf')],
                                       labels=['0-100', '101-200', '201-400', '401-600', '601-1000', '1000+'])

rate_by_range = mileage_rates.groupby('mileage_range')['implied_mileage_rate'].agg(['count', 'mean', 'std']).round(3)
print(f"\n=== MILEAGE RATES BY DISTANCE RANGE ===")
print(rate_by_range)


In [ ]:
# Visualize mileage patterns
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Scatter plot: Miles vs Reimbursement
axes[0,0].scatter(public_df['miles_traveled'], public_df['reimbursement'], alpha=0.6)
axes[0,0].set_xlabel('Miles Traveled')
axes[0,0].set_ylabel('Reimbursement ($)')
axes[0,0].set_title('Reimbursement vs Miles Traveled')

# Implied mileage rates by distance
mileage_rates.boxplot(column='implied_mileage_rate', by='mileage_range', ax=axes[0,1])
axes[0,1].set_title('Implied Mileage Rates by Distance Range')
axes[0,1].set_xlabel('Distance Range (Miles)')
axes[0,1].set_ylabel('Rate ($/mile)')

# Miles per day analysis (efficiency)
axes[1,0].scatter(public_df['miles_per_day'], public_df['reimbursement'], alpha=0.6, color='orange')
axes[1,0].set_xlabel('Miles per Day')
axes[1,0].set_ylabel('Reimbursement ($)')
axes[1,0].set_title('Reimbursement vs Efficiency (Miles/Day)')
axes[1,0].axvline(x=180, color='red', linestyle='--', label='180 miles/day')
axes[1,0].axvline(x=220, color='red', linestyle='--', label='220 miles/day')
axes[1,0].legend()

# Distribution of miles per day
axes[1,1].hist(public_df['miles_per_day'], bins=50, alpha=0.7, edgecolor='black')
axes[1,1].set_xlabel('Miles per Day')
axes[1,1].set_ylabel('Frequency')
axes[1,1].set_title('Distribution of Miles per Day')
axes[1,1].axvline(x=180, color='red', linestyle='--', label='180 miles/day')
axes[1,1].axvline(x=220, color='red', linestyle='--', label='220 miles/day')
axes[1,1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# Receipt analysis
print("=== RECEIPT ANALYSIS ===")

# Test the rounding bug theory (receipts ending in .49 or .99)
public_df['receipt_cents'] = (public_df['total_receipts_amount'] * 100) % 100
public_df['ends_in_49'] = public_df['receipt_cents'] == 49
public_df['ends_in_99'] = public_df['receipt_cents'] == 99
public_df['special_ending'] = public_df['ends_in_49'] | public_df['ends_in_99']

print("Rounding bug analysis:")
print(f"Cases ending in .49: {public_df['ends_in_49'].sum()}")
print(f"Cases ending in .99: {public_df['ends_in_99'].sum()}")
print(f"Cases with special endings: {public_df['special_ending'].sum()}")

if public_df['special_ending'].sum() > 0:
    special_avg = public_df[public_df['special_ending']]['reimbursement'].mean()
    normal_avg = public_df[~public_df['special_ending']]['reimbursement'].mean()
    print(f"Average reimbursement with special endings: ${special_avg:.2f}")
    print(f"Average reimbursement with normal endings: ${normal_avg:.2f}")
    print(f"Difference: ${special_avg - normal_avg:.2f}")

# Analyze receipt amounts by ranges
public_df['receipt_range'] = pd.cut(public_df['total_receipts_amount'], 
                                   bins=[0, 50, 100, 200, 500, 1000, 2000, float('inf')],
                                   labels=['0-50', '51-100', '101-200', '201-500', '501-1000', '1001-2000', '2000+'])

receipt_analysis = public_df.groupby('receipt_range').agg({
    'reimbursement': ['count', 'mean', 'std'],
    'total_receipts_amount': 'mean',
    'trip_duration_days': 'mean',
    'miles_traveled': 'mean'
}).round(2)

print(f"\n=== REIMBURSEMENT BY RECEIPT RANGE ===")
print(receipt_analysis)


In [ ]:
# Clustering analysis to identify different calculation paths
print("=== CLUSTERING ANALYSIS ===")

# Prepare features for clustering
features = ['trip_duration_days', 'miles_traveled', 'total_receipts_amount', 
           'miles_per_day', 'receipts_per_day']

# Create feature matrix
X = public_df[features].copy()

# Handle any infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Try different numbers of clusters
inertias = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    
    from sklearn.metrics import silhouette_score
    score = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(score)

# Plot elbow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(k_range, inertias, 'bo-')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia')
ax1.set_title('Elbow Method for Optimal k')

ax2.plot(k_range, silhouette_scores, 'ro-')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Silhouette Score')
ax2.set_title('Silhouette Score vs Number of Clusters')

plt.tight_layout()
plt.show()

# Use k=6 as suggested by Kevin's theory
optimal_k = 6
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
public_df['cluster'] = kmeans.fit_predict(X_scaled)

print(f"\nUsing {optimal_k} clusters as suggested by Kevin's theory:")
print(f"Silhouette score: {silhouette_score(X_scaled, public_df['cluster']):.3f}")

# Analyze clusters
cluster_analysis = public_df.groupby('cluster').agg({
    'trip_duration_days': ['count', 'mean', 'std'],
    'miles_traveled': ['mean', 'std'],
    'total_receipts_amount': ['mean', 'std'],
    'reimbursement': ['mean', 'std'],
    'miles_per_day': ['mean', 'std'],
    'receipts_per_day': ['mean', 'std']
}).round(2)

print(f"\n=== CLUSTER CHARACTERISTICS ===")
print(cluster_analysis)


In [ ]:
# Predictive modeling to understand feature importance
print("=== PREDICTIVE MODELING ===")

# Prepare features for modeling
model_features = ['trip_duration_days', 'miles_traveled', 'total_receipts_amount', 
                 'miles_per_day', 'receipts_per_day', 'receipt_to_mile_ratio']

# Add interaction features
public_df['trip_miles_interaction'] = public_df['trip_duration_days'] * public_df['miles_traveled']
public_df['trip_receipts_interaction'] = public_df['trip_duration_days'] * public_df['total_receipts_amount']
public_df['miles_receipts_interaction'] = public_df['miles_traveled'] * public_df['total_receipts_amount']

# Add categorical features as dummies
public_df['is_5_day_trip'] = (public_df['trip_duration_days'] == 5).astype(int)
public_df['is_short_trip'] = (public_df['trip_duration_days'] <= 2).astype(int)
public_df['is_long_trip'] = (public_df['trip_duration_days'] >= 8).astype(int)
public_df['is_high_efficiency'] = ((public_df['miles_per_day'] >= 180) & (public_df['miles_per_day'] <= 220)).astype(int)

extended_features = model_features + [
    'trip_miles_interaction', 'trip_receipts_interaction', 'miles_receipts_interaction',
    'is_5_day_trip', 'is_short_trip', 'is_long_trip', 'is_high_efficiency'
]

# Prepare data
X = public_df[extended_features].copy()
y = public_df['reimbursement'].copy()

# Handle missing/infinite values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest Model Performance:")
print(f"Mean Absolute Error: ${mae:.2f}")
print(f"R² Score: {r2:.3f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': extended_features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n=== FEATURE IMPORTANCE ===")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][::-1], feature_importance['importance'][::-1])
plt.xlabel('Feature Importance')
plt.title('Random Forest Feature Importance')
plt.tight_layout()
plt.show()


In [ ]:
# Summary of key findings
print("=== KEY FINDINGS SUMMARY ===")

print("\n1. TRIP DURATION PATTERNS:")
print(f"   - Most common trip lengths: {public_df['trip_duration_days'].value_counts().head(3).to_dict()}")
print(f"   - 5-day trip bonus confirmed: {five_day_rate:.2f} vs {avg_other_rate:.2f} per day")

print("\n2. MILEAGE PATTERNS:")
print(f"   - Average implied mileage rate: ${mileage_rates['implied_mileage_rate'].mean():.3f} per mile")
print(f"   - Rate varies by distance range (tiered structure confirmed)")
print(f"   - Efficiency sweet spot: 180-220 miles/day shows higher reimbursements")

print("\n3. RECEIPT PATTERNS:")
if public_df['special_ending'].sum() > 0:
    print(f"   - Rounding bug confirmed: {public_df['special_ending'].sum()} cases with .49/.99 endings")
    print(f"   - Special ending bonus: ${special_avg - normal_avg:.2f}")
else:
    print("   - No clear rounding bug pattern found in this dataset")

print("\n4. CLUSTERING INSIGHTS:")
print(f"   - {optimal_k} distinct calculation paths identified")
print(f"   - Clusters separate by trip characteristics and efficiency")

print("\n5. FEATURE IMPORTANCE:")
top_features = feature_importance.head(5)
for idx, row in top_features.iterrows():
    print(f"   - {row['feature']}: {row['importance']:.3f}")

print("\n=== IMPLEMENTATION RECOMMENDATIONS ===")
print("\n1. Base Structure:")
print("   - Start with ~$100/day base per diem")
print("   - Add 5-day trip bonus")
print("   - Implement tiered mileage rates")

print("\n2. Efficiency Bonuses:")
print("   - Reward 180-220 miles/day range")
print("   - Consider trip_duration * miles_traveled interaction")

print("\n3. Receipt Processing:")
print("   - Implement diminishing returns for high amounts")
print("   - Check for rounding bugs with .49/.99 endings")
print("   - Consider receipt penalties for very low amounts")

print("\n4. Special Cases:")
print("   - Handle edge cases for very short/long trips")
print("   - Implement cluster-specific adjustments")

print("\n=== NEXT STEPS ===")
print("1. Implement basic calculation in run.sh")
print("2. Test against public cases using eval.sh")
print("3. Iteratively refine based on error analysis")
print("4. Focus on high-error cases to identify missing patterns")
print("5. Validate final model against private cases")


In [ ]:
# Load the public cases data
with open('public_cases.json', 'r') as f:
    public_data = json.load(f)

# Convert to DataFrame
public_df = pd.DataFrame([
    {
        'trip_duration_days': case['input']['trip_duration_days'],
        'miles_traveled': case['input']['miles_traveled'],
        'total_receipts_amount': case['input']['total_receipts_amount'],
        'reimbursement': case['expected_output']
    }
    for case in public_data
])

# Load private cases (inputs only)
with open('private_cases.json', 'r') as f:
    private_data = json.load(f)

private_df = pd.DataFrame([
    {
        'trip_duration_days': case['trip_duration_days'],
        'miles_traveled': case['miles_traveled'],
        'total_receipts_amount': case['total_receipts_amount']
    }
    for case in private_data
])

print(f"Public dataset: {len(public_df)} cases")
print(f"Private dataset: {len(private_df)} cases")
print("\nPublic data sample:")
print(public_df.head())
